In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats
from scipy.stats.mstats import winsorize
import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_info_rows', 2000)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.width', 2000)
sns.set_style("whitegrid")
#baslik_font = {'family': 'arial', 'color': 'darkred','weight': 'bold','size': 13 }
#eksen_font  = {'family': 'arial', 'color': 'darkblue','weight': 'bold','size': 11 }
pd.options.display.float_format = '{:.2f}'.format

In [2]:
df = pd.read_csv("clean_scout_20200923.csv")

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

In [4]:
df.shape

(15919, 33)

In [5]:
df.head(3)

,make_model,body_type,price,vat,km,Type,Warranty,Cylinders,Fuel,Gears,Comfort&Convenience,Entertainment&Media,Extras,Safety&Security,Previous_Owners,hp_kW,Inspection_new,age,Body_Color,Paint_Type,Upholstery_type,Upholstery_color,Nr_of_Doors,Nr_of_Seats,Gearing_Type,Displacement_cc,Weight_kg,Drive_chain,cons_comb,cons_city,cons_country,CO2_Emission,Emission_Class
0,Audi A1,Sedans,15770,VAT deductible,56013.00,Used,nan,3.00,Diesel,nan,"Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Light sensor,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system","Bluetooth,Hands-free equipment,On-board computer,Radio","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Xenon headlights",2.00,66.00,Yes,3.00,Black,Metallic,Cloth,Black,5.00,5.00,Automatic,1422.00,1220.00,front,3.80,4.30,3.50,99.00,Euro 6
1,Audi A1,Sedans,14500,Price negotiable,80000.00,Used,nan,4.00,Benzine,7.00,"Air conditioning,Automatic climate control,Hill Holder,Leather steering wheel,Lumbar support,Parking assist system sensors rear,Power windows,Start-stop system,Tinted windows","Bluetooth,Hands-free equipment,On-board computer,Radio,Sound system","Alloy wheels,Sport seats,Sport suspension,Voice Control","ABS,Central door lock,Central door lock with remote control,Daytime running lights,Driver-side airbag,Electronic stability control,Head airbag,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Xenon headlights",nan,141.00,NaN,2.00,Red,NaN,Cloth,Grey,3.00,4.00,Automatic,1798.00,1255.00,front,5.60,7.10,4.70,129.00,Euro 6
2,Audi A1,Sedans,14640,VAT deductible,83450.00,Used,nan,nan,Diesel,nan,"Air conditioning,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Seat heating,Start-stop system","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",1.00,85.00,NaN,3.00,Black,Metallic,Cloth,Black,4.00,4.00,Automatic,1598.00,nan,front,3.80,4.40,3.40,99.00,Euro 6


In [6]:
df.isnull().sum()/df.shape[0]*100

make_model             0.00
body_type              0.38
price                  0.00
vat                   28.35
km                     6.43
Type                   0.01
Warranty              69.51
Cylinders             35.68
Fuel                   0.00
Gears                 29.60
Comfort&Convenience    5.78
Entertainment&Media    8.63
Extras                18.61
Safety&Security        6.17
Previous_Owners       41.71
hp_kW                  0.55
Inspection_new        75.30
age                   10.03
Body_Color             3.75
Paint_Type            36.26
Upholstery_type        0.00
Upholstery_color       0.00
Nr_of_Doors            1.33
Nr_of_Seats            6.14
Gearing_Type           0.00
Displacement_cc        3.12
Weight_kg             43.81
Drive_chain           43.08
cons_comb             12.77
cons_city             15.30
cons_country          14.93
CO2_Emission          15.30
Emission_Class        22.79
dtype: float64

In [7]:
miss_val = []

[miss_val.append(i) for i in df.columns if any(df[i].isnull())]

miss_val

['body_type',
 'vat',
 'km',
 'Type',
 'Warranty',
 'Cylinders',
 'Gears',
 'Comfort&Convenience',
 'Entertainment&Media',
 'Extras',
 'Safety&Security',
 'Previous_Owners',
 'hp_kW',
 'Inspection_new',
 'age',
 'Body_Color',
 'Paint_Type',
 'Nr_of_Doors',
 'Nr_of_Seats',
 'Displacement_cc',
 'Weight_kg',
 'Drive_chain',
 'cons_comb',
 'cons_city',
 'cons_country',
 'CO2_Emission',
 'Emission_Class']

## functions to fill the missing values

In [8]:
def fill_most(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        grp_inx = list(df[cond][col_name].index)
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[cond][col_name].mode()[0])
        else:
            df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [9]:
def fill_prop(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        grp_inx = list(df[cond][col_name].index)
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [10]:
def fill(df, group_col1, group_col2, col_name, method): # method can be "mode" or "median" or "ffill"
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                grp_inx = list(df[cond2][col_name].index)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if (mode1 != []) and (mode2 != []):
                    df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[cond2][col_name].mode()[0]).fillna(df[cond1][col_name].mode()[0])
                elif mode1 != []:
                    df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[cond1][col_name].mode()[0])
                else:
                    df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[col_name].mode()[0])
                
    elif method == "median":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                grp_inx = list(df[cond2][col_name].index)
                df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[cond2][col_name].median()).fillna(df[cond1][col_name].median()).fillna(df[col_name].median())
                
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                grp_inx = list(df[cond2][col_name].index)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

## age

In [11]:
df["age"].value_counts(dropna=False)

1.00    4522
3.00    3674
2.00    3273
0.00    2853
nan     1597
Name: age, dtype: int64

In [12]:
df['age'].fillna('-', inplace=True)

In [13]:
df.groupby('age').km.mean()

age
0.0    2085.36
1.0   18035.24
2.0   41754.94
3.0   77442.52
-       934.50
Name: km, dtype: float64

In [14]:
df['age'].replace('-',0, inplace=True)

In [15]:
df["age"].value_counts(dropna=False)

1.00    4522
0.00    4450
3.00    3674
2.00    3273
Name: age, dtype: int64

In [16]:
df.groupby('age').km.mean()

age
0.00    1833.26
1.00   18035.24
2.00   41754.94
3.00   77442.52
Name: km, dtype: float64

## km

In [17]:
df.km.value_counts(dropna=False)

10.00       1045
nan         1024
1.00         367
5.00         170
50.00        148
            ... 
8329.00        1
267.00         1
1060.00        1
73652.00       1
49320.00       1
Name: km, Length: 6690, dtype: int64

In [18]:
df.groupby("age").km.transform("mean")

0       77442.52
1       41754.94
2       77442.52
3       77442.52
4       77442.52
          ...   
15914    1833.26
15915    1833.26
15916    1833.26
15917    1833.26
15918    1833.26
Name: km, Length: 15919, dtype: float64

In [19]:
df["km"].fillna(df.groupby("age").km.transform("mean"), inplace=True)

In [20]:
df.km.value_counts(dropna=False)

10.00       1045
1833.26      985
1.00         367
5.00         170
50.00        148
            ... 
8329.00        1
267.00         1
1060.00        1
73652.00       1
15577.00       1
Name: km, Length: 6692, dtype: int64

## Previous_Owners

In [21]:
df["Previous_Owners"].value_counts(dropna=False)

1.00    8294
nan     6640
2.00     778
0.00     188
3.00      17
4.00       2
Name: Previous_Owners, dtype: int64

In [22]:
df["Previous_Owners"].fillna("-", inplace = True)

In [23]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
-      6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [24]:
df.groupby(['make_model', 'Previous_Owners']).km.describe()

count     mean      std      min      25%      50%      75%       max
make_model     Previous_Owners                                                                        
Audi A1        0.0               51.00  7130.76 17874.36     0.00    10.00  1833.26  1833.26  68002.00
               1.0             1331.00 25460.48 26306.05     1.00  6389.50 16885.00 34600.00 192000.00
               2.0              171.00 34146.70 22186.26  3000.00 17768.00 27300.00 49351.00 129550.00
               3.0                2.00 31945.00 18462.56 18890.00 25417.50 31945.00 38472.50  45000.00
               4.0                1.00 19700.00      nan 19700.00 19700.00 19700.00 19700.00  19700.00
               -               1058.00 16174.83 23076.47     1.00  1833.26  5500.00 22595.25 146140.00
Audi A2        1.0                1.00 26166.00      nan 26166.00 26166.00 26166.00 26166.00  26166.00
Audi A3        0.0               55.00  4678.54 11815.26     0.00    10.00  1833.26  1833.26  64500.00
               1.0             1446.00 46460.36 45361.62     1.00  9800.00 28869.50 79150.00 291800.00
               2.0              111.00 46306.54 30725.25    15.00 22447.50 35000.00 69121.00 138700.00
               3.0                2.00 71800.00 62508.24 27600.00 49700.00 71800.00 93900.00 116000.00
               -               1483.00 31141.84 36912.10     1.00   300.00 19500.00 47200.00 217032.00
Opel Astra     0.0               13.00  2939.54  8415.04     0.00     5.00    10.00  1833.26  30800.00
               1.0             1487.00 42902.07 42474.83     2.00 11796.00 28146.00 67186.50 260000.00
               2.0              134.00 46466.74 36370.69   800.00 21379.00 34685.50 59344.25 229500.00
               3.0                3.00 60733.00  3763.03 56500.00 59250.00 62000.00 62849.50  63699.00
               -                889.00 25451.71 34453.88     1.00   700.00  8495.00 40000.00 248000.00
Opel Corsa     0.0               19.00 19072.70 29220.27     0.00     5.50  1833.26 25190.00  90509.00
               1.0             1326.00 23558.37 31785.60     1.00    51.00 12038.00 31238.50 317000.00
               2.0              138.00 33912.76 21872.65    10.00 19688.25 31386.00 43922.00 116184.00
               3.0                2.00 23445.00  8563.06 17390.00 20417.50 23445.00 26472.50  29500.00
               -                734.00 25618.58 29849.21     1.00   250.00 17026.50 39959.25 179521.00
Opel Insignia  0.0               18.00  2938.10  9293.08     0.00     5.50    10.00  1833.26  40000.00
               1.0             1509.00 44092.62 42622.74     1.00 16389.00 26886.00 68920.00 232000.00
               2.0               93.00 60272.69 40132.47  7120.00 33000.00 48500.00 77000.00 200000.00
               3.0                4.00 44606.25 15445.45 32300.00 35918.75 39562.50 48250.00  67000.00
               -                974.00 27913.55 37619.25     1.00  1833.26  7632.50 42000.00 214999.00
Renault Clio   0.0               26.00  5431.02 17903.38     0.00    10.00    20.00  1833.26  86000.00
               1.0              800.00 28982.27 33478.68     1.00  3753.75 18900.00 37000.00 161556.00
               2.0               70.00 34931.99 20311.15    15.00 20167.75 42000.00 49178.00  93000.00
               3.0                2.00 62500.00  4949.75 59000.00 60750.00 62500.00 64250.00  66000.00
               -                941.00 29430.81 32358.99     1.00  1833.26 22000.00 44700.00 163500.00
Renault Duster 0.0                1.00  1833.26      nan  1833.26  1833.26  1833.26  1833.26   1833.26
               -                 33.00   993.38   879.16   101.00   101.00  1833.26  1833.26   1833.26
Renault Espace 0.0                5.00 18765.25 40881.48     0.00    10.00   100.00  1833.26  91883.00
               1.0              394.00 33854.33 38917.16     1.00  1833.26 18035.24 55000.00 197414.00
               2.0               61.00 45397.39 20974.10    50.00 38000.00 49606.00 49606.00 148000.00
               3

In [25]:
df[(df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")]["km"]

14894   1833.26
14895   1833.26
14896    101.00
14897   1833.26
14898    101.00
14899    101.00
14900   1833.26
14901    101.00
14903    101.00
14904   1833.26
14905   1833.26
14906    101.00
14907    101.00
14908   1833.26
14909   1833.26
14910    101.00
14911   1833.26
14912    101.00
14913   1833.26
14914    101.00
14915    101.00
14916    101.00
14917   1833.26
14918   1833.26
14919   1833.26
14920    101.00
14921   1833.26
14922   1833.26
14923    101.00
14924   1833.26
14925    101.00
14926    101.00
14927   1833.26
Name: km, dtype: float64

In [26]:
con = (df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "-")
df.loc[con, "Previous_Owners"] = df.loc[con, "Previous_Owners"].replace("-", 0.0)

In [27]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
-      6607
2.0     778
0.0     221
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [28]:
df["Previous_Owners"].replace("-", np.nan, inplace=True)

In [29]:
fill_prop(df, "age", "Previous_Owners")

Number of NaN :  0
------------------
1.00    14163
2.00     1170
0.00      555
3.00       29
4.00        2
Name: Previous_Owners, dtype: int64


In [30]:
#df["Previous_Owners"].fillna(method='ffill', inplace=True)

In [31]:
df["Previous_Owners"].value_counts(dropna=False)

1.00    14163
2.00     1170
0.00      555
3.00       29
4.00        2
Name: Previous_Owners, dtype: int64

## Warranty

##### çok fazla nan value var ve bu nan değerlerin km, yaş ve araba model sütunlarına göre incelemesini yapınca bu sütunun sağlıklı bir veriye sahip olmadığına karar verdik

In [32]:
df.drop("Warranty", axis=1, inplace=True)

## price

In [33]:
#df[df["price"]<3000].price.value_counts()

In [34]:
#df = df[df["price"]>3000]

## vat

In [35]:
df.vat.fillna(method="ffill", inplace = True)

In [36]:
df.vat.value_counts(dropna=False)

VAT deductible      15048
Price negotiable      871
Name: vat, dtype: int64

## Body Color

In [37]:
fill(df, "make_model", "body_type", "Body_Color", "ffill")

Number of NaN :  0
------------------
Black     3901
Grey      3620
White     3514
Silver    1710
Blue      1515
Red        995
Brown      298
Green      166
Beige      120
Yellow      51
Violet      18
Bronze       6
Orange       3
Gold         2
Name: Body_Color, dtype: int64


In [38]:
#df["Body_Color"].fillna(method="ffill", inplace=True)

### Type

In [39]:
df['Type'].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: Type, dtype: int64

In [40]:
df.Type.fillna("-", inplace=True)

In [41]:
df['Type'].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
-                     2
Name: Type, dtype: int64

In [42]:
df.groupby(["Type", "make_model", "age"]).km.describe()

count      mean      std       min       25%       50%       75%       max
Type           make_model     age                                                                             
-              Audi A3        0.00   1.00   1833.26      nan   1833.26   1833.26   1833.26   1833.26   1833.26
                              3.00   1.00 115137.00      nan 115137.00 115137.00 115137.00 115137.00 115137.00
Demonstration  Audi A1        0.00 111.00   3851.04  2584.72     10.00   2582.50   3001.00   5000.00  12700.00
                              1.00  18.00   9860.67  5351.24   1050.00   5200.00   9261.00  14560.00  18900.00
                              2.00   6.00  22963.50 17802.68  11940.00  15327.25  16378.50  18425.00  59000.00
               Audi A3        0.00  41.00   4971.88  3092.35     50.00   3000.00   5000.00   6000.00  14000.00
                              1.00  16.00  12801.19 10920.15   1015.00   4562.25   9850.00  21020.75  38400.00
                              2.00   5.00  27950.00  9145.35  16000.00  23000.00  27500.00  33750.00  39500.00
               Opel Astra     0.00 148.00   2905.94  2873.94      3.00    315.25   1866.63   4999.00  13542.00
                              1.00  16.00   7319.94  7499.40    100.00   2160.50   2499.00  13514.75  22700.00
                              2.00   1.00  68500.00      nan  68500.00  68500.00  68500.00  68500.00  68500.00
               Opel Corsa     0.00  75.00   1271.30  1693.67      3.00     10.00    518.00   1833.26   7623.00
                              1.00  14.00   2915.50  5334.49     10.00    100.00    200.00   3157.50  18000.00
               Opel Insignia  0.00 162.00   3989.44  4004.63      5.00   1000.00   3000.00   5555.00  20000.00
                              1.00   5.00  14265.20 12475.15   3000.00   4000.00   9206.00  25200.00  29920.00
                              2.00   4.00  27175.00  4251.57  23300.00  24500.00  26200.00  28875.00  33000.00
                              3.00   1.00  56000.00      nan  56000.00  56000.00  56000.00  56000.00  56000.00
               Renault Clio   0.00  39.00    654.61  1224.26      5.00      7.00     10.00    500.00   5000.00
                              1.00  33.00   5555.85  6384.85      8.00   1417.00   3500.00   6523.00  24000.00
                              2.00   3.00  10014.00  9124.49     20.00   6071.00  12122.00  15011.00  17900.00
                              3.00   1.00  35000.00      nan  35000.00  35000.00  35000.00  35000.00  35000.00
               Renault Espace 0.00  61.00   4422.24  5072.35      5.00   1000.00   2860.00   5000.00  19900.00
                              1.00  28.00  10118.93  7526.32     10.00   4250.00  10592.50  16113.25  30000.00
                              2.00   8.00  24883.00 22532.02   7000.00  13925.00  17820.00  24905.00  78000.00
Employee's car Audi A1        0.00  53.00   4510.12  4374.27      1.00    665.00   4500.00   6900.00  18000.00
                              1.00 131.00  12199.40  7558.41      1.00   6319.50  10000.00  18475.00  28319.00
                              2.00   8.00  17377.75  5795.69   9767.00  11816.75  18900.00  20795.50  24400.00
                              3.00   5.00  48103.80 30327.71  26792.00  27000.00  27000.00  67278.00  92449.00
               Audi A2        1.00   1.00  26166.00      nan  26166.00  26166.00  26166.00  26166.00  26166.00
               Audi A3        0.00  31.00   3869.45  5402.69     10.00    100.00    500.00   5900.00  18000.00
                              1.00 157.00  18197.57  8303.33      1.00  12000.00  19440.00  24500.00  35970.00
                              2.00  32.00  31584.75 14949.53     59.00  23750.00  32000.00  41959.00  60000.00
                              3.00   8.00  56954.62 26796.57   7790.00  40700.00  58481.00  81095.75  85100.00
               Opel Astra     0.00  18.00   5451.06  2971.35    431.00   4355.25   5957.50   7413.25  11707.00
                              1.00 121.0

In [43]:
cond1 = (df['make_model'] == "Audi A3") & (df["age"] == 0)
cond2 = (df['make_model'] == "Audi A3") & (df["age"] == 3)

In [44]:
df.loc[cond1,'Type'] = df.loc[cond1,'Type'].replace('-','New')
df.loc[cond2,'Type'] = df.loc[cond2,'Type'].replace('-','Used')

In [45]:
df['Type'].value_counts(dropna=False)

Used              11097
New                1651
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: Type, dtype: int64

## body type

In [46]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [47]:
df.body_type.replace("Other", np.nan, inplace=True) 

In [48]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
NaN               350
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [49]:
df['body_type'] = df['body_type'].fillna('-')

In [50]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
-                 350
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [51]:
# df.groupby(['make_model','body_type', 'Nr. of Doors']).price.count()
# her araba modeli ve kapı sayısına göre body_type belirledik.

con1 = df['make_model'] == 'Audi A3' # coupe
con2 = df['make_model'] == 'Opel Astra' # Station wagon
con3 = df['make_model'] == 'Opel Corsa' # Sedan
con4 = df['make_model'] == 'Opel Insignia' # Station wagon
con5 = df['make_model'] == 'Renault Clio' # Sedan
con6 = df['make_model'] == 'Renault Espace' # Van

df.loc[con1,'body_type'] = df.loc[con1,'body_type'].replace('-','Coupe')
df.loc[con2,'body_type'] = df.loc[con2,'body_type'].replace('-','Station wagon')
df.loc[con3,'body_type'] = df.loc[con3,'body_type'].replace('-','Sedans')
df.loc[con4,'body_type'] = df.loc[con4,'body_type'].replace('-','Station wagon')
df.loc[con5,'body_type'] = df.loc[con5,'body_type'].replace('-','Sedans')
df.loc[con6,'body_type'] = df.loc[con6,'body_type'].replace('-','Van')

In [52]:
df.body_type.replace("-", np.nan, inplace=True)

In [53]:
fill_most(df, "make_model", "body_type")

Number of NaN :  0
------------------
Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


## Inspection new

In [54]:
df["Inspection_new"].value_counts(dropna=False)

NaN    11987
Yes     3932
Name: Inspection_new, dtype: int64

In [55]:
df["Inspection_new"].fillna("No", inplace=True)

In [56]:
df["Inspection_new"].value_counts(dropna=False)

No     11987
Yes     3932
Name: Inspection_new, dtype: int64

In [57]:
df.groupby(["make_model","age", "Inspection_new"]).price.describe()

count     mean     std      min      25%      50%      75%      max
make_model     age  Inspection_new                                                                     
Audi A1        0.00 No             610.00 24156.69 3353.54 15679.00 21500.00 23600.00 26997.50 37900.00
                    Yes            199.00 22864.30 3348.79 14900.00 20404.00 22400.00 24935.00 29197.00
               1.00 No             477.00 18776.50 2701.39 13450.00 16500.00 18500.00 20940.00 33900.00
                    Yes            267.00 17879.94 2329.99 13980.00 16261.00 16871.00 19882.50 23880.00
               2.00 No             333.00 16622.27 2178.82 10999.00 15250.00 15888.00 17970.00 23490.00
                    Yes             99.00 16637.39 2255.68 12490.00 14965.00 15860.00 17989.00 21490.00
               3.00 No             406.00 14502.52 1888.22  8999.00 13400.00 14000.00 15497.50 19900.00
                    Yes            223.00 14466.55 1896.89  9950.00 12990.00 13950.00 15869.50 18880.00
Audi A2        1.00 No               1.00 28200.00     nan 28200.00 28200.00 28200.00 28200.00 28200.00
Audi A3        0.00 No             648.00 25074.89 3227.22 15500.00 23525.00 24900.00 26262.50 63900.00
                    Yes            179.00 24379.34 1722.48 19078.00 23345.00 24490.00 25487.00 27488.00
               1.00 No             693.00 22625.44 5546.26  5555.00 20800.00 21500.00 22900.00 74600.00
                    Yes             84.00 21505.23 1274.74 17996.00 20810.00 21461.00 22703.75 23770.00
               2.00 No             573.00 20064.66 4261.77 10550.00 18100.00 19334.00 20900.00 67600.00
                    Yes            102.00 19236.79 1884.13 14960.00 17855.00 18980.00 21335.00 21999.00
               3.00 No             664.00 16583.39 2975.79 10950.00 15000.00 15950.00 17890.00 42500.00
                    Yes            154.00 15960.35 1737.70 10990.00 14920.00 15930.00 17485.00 18790.00
Opel Astra     0.00 No             450.00 20881.24 5325.30  6950.00 17335.00 18490.00 24842.50 39433.00
                    Yes            243.00 21985.76 4533.79 13945.00 18095.00 20715.00 24969.50 34900.00
               1.00 No             503.00 15666.22 1902.69   255.00 14690.00 15150.00 16994.50 18990.00
                    Yes            224.00 17186.43 1615.81 12489.00 15450.00 17865.00 18490.00 18990.00
               2.00 No             433.00 13118.71 2111.69  7800.00 11700.00 12636.00 14900.00 19950.00
                    Yes            154.00 12700.33 1709.85  9890.00 11592.25 12460.00 12980.00 17890.00
               3.00 No             341.00 11086.26 2477.43  6000.00  9490.00 10500.00 12290.00 17750.00
                    Yes            178.00 11869.23 2474.35  7280.00 10051.00 10950.00 14472.50 15999.00
Opel Corsa     0.00 No             406.00 13281.30 2865.79   120.00 10990.00 13275.00 15650.00 18622.00
                    Yes            319.00 13612.12 2601.73  6490.00 10979.00 14213.00 15950.00 18490.00
               1.00 No             527.00 10801.40 2033.01    13.00  9495.00  9900.00 12697.50 15690.00
                    Yes            156.00 13023.58 1388.06  8880.00 12835.00 13445.00 13980.00 15790.00
               2.00 No             318.00  9024.62 1473.40  6200.00  8300.00  8700.00  8950.00 13990.00
                    Yes             64.00 10433.39 1862.13  6950.00  8690.00 10985.00 11950.00 13880.00
               3.00 No             299.00  8168.22 1859.88  4950.00  7300.00  7750.00  7950.00 16950.00
                    Yes            130.00  8270.40 1661.91  6290.00  7390.00  7625.00  7937.50 13450.00
Opel Insignia  0.00 No             530.00 31888.96 5636.68 11980.00 27828.00 32419.50 36267.50 49884.00
                    Yes            145.00 32796.48 5032.17 24315.00 28615.00 33450.00 36391.00 52120.00
               1.00 No             457.00 20927.27 2039.77 15990.00 19800.00 20900.00 22550.00 24590.00
                    Yes            310.00 21983.47 1833.01 16980.00 20757.50 21950.0

In [58]:
df["Inspection_new"].replace(["Yes", "No"], [1,0], inplace = True)

In [59]:
df["Inspection_new"].value_counts(dropna=False)

0    11987
1     3932
Name: Inspection_new, dtype: int64

### Paint Type

In [60]:
df["Paint_Type"].value_counts(dropna=False)

Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: Paint_Type, dtype: int64

In [61]:
fill(df, "make_model", "body_type", "Paint_Type", "ffill")

Number of NaN :  0
------------------
Metallic       15250
Uni/basic        637
Perl effect       32
Name: Paint_Type, dtype: int64


In [62]:
#df["Paint_Type"].fillna(method = "ffill", inplace=True)

In [63]:
df["Paint_Type"].value_counts(dropna=False)

Metallic       15250
Uni/basic        637
Perl effect       32
Name: Paint_Type, dtype: int64

## Upholstery_type

In [64]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth           8423
Other           4871
Part leather    1499
Full leather    1009
Velour            60
alcantara         57
Name: Upholstery_type, dtype: int64

In [65]:
df["Upholstery_type"].replace(["Other","Velour", "alcantara", "Part leather", "Full leather"], [np.nan, "Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"], inplace=True)

In [66]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth                8483
NaN                  4871
Part/Full Leather    2565
Name: Upholstery_type, dtype: int64

In [67]:
fill(df, "make_model", "body_type", "Upholstery_type", "ffill")

Number of NaN :  0
------------------
Cloth                12238
Part/Full Leather     3681
Name: Upholstery_type, dtype: int64


In [68]:
#Alternatif çözüm

#oto1["upholstery"].str.contains("leather|alcantara", case = False, regex = True).value_counts(dropna = False)
#oto1["upholstery"].str.contains("leather|alcantara", case = False, regex = True).value_counts().sum()

#Buradan elde edilen değerlere göre veriler arasındaki oran bulundu. Ve bu orana göre missing value'lar dolduruldu.

#oto1.upholstery.fillna("Cloth", limit = 1000, inplace = True)
#oto1.upholstery.fillna("Leather", limit = 410, inplace = True)
#oto1.upholstery.fillna("Cloth", limit = 1000, inplace = True)
#oto1.upholstery.fillna("Leather", limit = 410, inplace = True)
#oto1.upholstery.fillna("Cloth", limit = 900, inplace = True)

#upholstery_bool_list = oto1['upholstery'].str.contains('leather|alcantara', case = False, regex = True)
#oto1.loc[upholstery_bool_list, "upholstery"] = "Part/Full Leather"
#oto1.upholstery.loc[lambda x : x!= "Part/Full Leather"] = "Cloth"

## Upholstery_color

In [69]:
df.drop("Upholstery_color", axis=1, inplace=True)

### Nr. of Doors

In [70]:
df["Nr_of_Doors"].value_counts(dropna=False)

5.00    11575
4.00     3079
3.00      832
2.00      219
nan       212
7.00        1
1.00        1
Name: Nr_of_Doors, dtype: int64

In [71]:
fill(df, "make_model", "body_type", "Nr_of_Doors", "mode")

Number of NaN :  0
------------------
5.00    11787
4.00     3079
3.00      832
2.00      219
7.00        1
1.00        1
Name: Nr_of_Doors, dtype: int64


In [72]:
#df["Nr_of_Doors"].fillna(method = "ffill", inplace=True)

In [73]:
df["Nr_of_Doors"].value_counts(dropna=False)

5.00    11787
4.00     3079
3.00      832
2.00      219
7.00        1
1.00        1
Name: Nr_of_Doors, dtype: int64

### Nr. of Seats

In [74]:
df["Nr_of_Seats"].value_counts(dropna=False)

5.00    13336
4.00     1125
nan       977
7.00      362
2.00      116
6.00        2
3.00        1
Name: Nr_of_Seats, dtype: int64

In [75]:
fill(df, "make_model", "body_type", "Nr_of_Seats", "mode")

Number of NaN :  0
------------------
5.00    14308
4.00     1127
7.00      362
2.00      119
6.00        2
3.00        1
Name: Nr_of_Seats, dtype: int64


In [76]:
#df["Nr_of_Seats"].fillna(method = "ffill", inplace=True)

In [77]:
df["Nr_of_Seats"].value_counts(dropna=False)

5.00    14308
4.00     1127
7.00      362
2.00      119
6.00        2
3.00        1
Name: Nr_of_Seats, dtype: int64

### Cylinders

In [78]:
df["Cylinders"].value_counts(dropna=False)

4.00    8105
nan     5680
3.00    2104
5.00      22
6.00       3
2.00       2
8.00       2
1.00       1
Name: Cylinders, dtype: int64

In [79]:
fill(df, "make_model", "body_type", "Cylinders", "median")

Number of NaN :  0
------------------
4.00    12926
3.00     2963
5.00       22
6.00        3
2.00        2
8.00        2
1.00        1
Name: Cylinders, dtype: int64


In [80]:
#df['Cylinders'] = df['Cylinders'].fillna(df.groupby('make_model')['Cylinders'].transform('median'))

In [81]:
df["Cylinders"].value_counts(dropna=False)

4.00    12926
3.00     2963
5.00       22
6.00        3
2.00        2
8.00        2
1.00        1
Name: Cylinders, dtype: int64

### Drive chain

In [82]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
NaN      6858
4WD       171
rear        4
Name: Drive_chain, dtype: int64

In [83]:
df["Drive_chain"].fillna("-", inplace=True)

In [84]:
df.groupby(["make_model", "body_type", "Drive_chain"]).price.describe()

count     mean      std      min      25%      50%      75%      max
make_model     body_type     Drive_chain                                                                       
Audi A1        Compact       -            352.00 17620.87  4226.12 10490.00 14990.00 15900.00 20885.75 29190.00
                             4WD            2.00 14790.00  1258.65 13900.00 14345.00 14790.00 15235.00 15680.00
                             front        685.00 20008.22  4511.35  9950.00 16430.00 19890.00 22690.00 31990.00
               Coupe         -              2.00 14925.00  1378.86 13950.00 14437.50 14925.00 15412.50 15900.00
               Sedans        -            561.00 17830.44  4362.32  8999.00 14900.00 16490.00 20700.00 37900.00
                             4WD            1.00 15450.00      nan 15450.00 15450.00 15450.00 15450.00 15450.00
                             front        989.00 19133.79  4441.97 10000.00 15838.00 18500.00 21999.00 32000.00
               Station wagon -              3.00 24593.00  7537.22 15890.00 22390.00 28890.00 28944.50 28999.00
                             front         18.00 16681.11  2493.67 12950.00 15000.00 16356.00 17300.00 21450.00
               Van           front          1.00 29000.00      nan 29000.00 29000.00 29000.00 29000.00 29000.00
Audi A2        Off-Road      front          1.00 28200.00      nan 28200.00 28200.00 28200.00 28200.00 28200.00
Audi A3        Compact       -             25.00 18236.52  3427.24 12900.00 16400.00 17894.00 18700.00 29950.00
                             4WD            1.00 67600.00      nan 67600.00 67600.00 67600.00 67600.00 67600.00
                             front        156.00 18839.37  6078.74 13100.00 13910.00 17724.50 20810.00 41495.00
               Convertible   -              4.00 26112.50  6123.78 20000.00 21500.00 25750.00 30362.50 32950.00
                             4WD            1.00 56100.00      nan 56100.00 56100.00 56100.00 56100.00 56100.00
                             front          3.00 20897.67 10003.50 10893.00 15896.50 20900.00 25900.00 30900.00
               Coupe         -              1.00 16990.00      nan 16990.00 16990.00 16990.00 16990.00 16990.00
                             front          3.00 17996.67  1005.00 16990.00 17495.00 18000.00 18500.00 19000.00
               Sedans        -            891.00 20787.71  5846.56 10550.00 17000.00 19900.00 22990.00 64900.00
                             4WD           17.00 32913.76 20353.18 13780.00 16999.00 22499.00 51999.00 74600.00
                             front       1713.00 21223.41  3546.98 10990.00 18700.00 21490.00 23800.00 34738.00
               Station wagon -            149.00 20254.74  5473.08  5555.00 16999.00 19800.00 23401.00 63900.00
                             front        133.00 21165.65  3552.00 13250.00 18480.00 21450.00 24200.00 27400.00
Opel Astra     Compact       -             81.00 17993.35  6740.20  9490.00 14900.00 15400.00 17800.00 37995.00
                             front        104.00 14802.84  5080.91  8450.00 11955.00 12955.00 16344.00 39433.00
               Coupe         -              2.00 22240.00 10960.16 14490.00 18365.00 22240.00 26115.00 29990.00
               Off-Road      front          1.00 18590.00      nan 18590.00 18590.00 18590.00 18590.00 18590.00
               Sedans        -            507.00 15488.44  4123.97  6500.00 12962.00 15249.00 16990.00 30750.00
                             4WD            2.00 13075.00  3641.60 10500.00 11787.50 13075.00 14362.50 15650.00
                             front        544.00 16416.24  4600.63  8860.00 12900.00 15890.00 18457.50 32950.00
               Station wagon -            588.00 14923.39  4925.89   255.00 10900.00 14895.00 17714.00 33995.00
                             front        697.00 16290.03  5318.15  6690.00 12250.00 15890.00 18490.00 34900.00
Opel Corsa     Compact       -            678.00 10877.92  2906.95   120.00  8750.00  9950.00 12990.00 18501.00
           

In [85]:
condi = (df['make_model'] == "Renault Duster") & (df["body_type"] == "Off-Road")

In [86]:
df.loc[condi,'Drive_chain'] = df.loc[condi,'Drive_chain'].replace('-','4WD')

In [87]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
-        6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [88]:
df["Drive_chain"] = df["Drive_chain"].replace('-', np.nan)

In [89]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
NaN      6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [90]:
fill(df, "make_model", "body_type", "Drive_chain", "mode")

Number of NaN :  0
------------------
front    15711
4WD        204
rear         4
Name: Drive_chain, dtype: int64


In [91]:
#df["Drive_chain"].fillna(method = "ffill", inplace=True)

In [92]:
df["Drive_chain"].value_counts(dropna=False)

front    15711
4WD        204
rear         4
Name: Drive_chain, dtype: int64

### Emission Class

In [93]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [94]:
df["Emission_Class"].fillna("-", inplace=True)

In [95]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
-          3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [96]:
df.groupby(["make_model", "age", "Fuel", "Emission_Class"]).price.describe()

count     mean      std      min      25%      50%      75%      max
make_model     age  Fuel     Emission_Class                                                                      
Audi A1        0.00 Benzine  -              268.00 23747.29  3153.21 14900.00 21850.00 22900.00 26315.00 31990.00
                             Euro 6         535.00 23948.53  3472.58 15550.00 21285.00 22900.00 27107.50 37900.00
                    Diesel   -                1.00 15679.00      nan 15679.00 15679.00 15679.00 15679.00 15679.00
                             Euro 6           5.00 18632.00  1941.87 15680.00 17900.00 18990.00 20000.00 20590.00
               1.00 Benzine  -               98.00 19334.58  2864.66 14490.00 16676.75 19893.50 21900.00 23880.00
                             Euro 5           3.00 17862.67  1840.59 16800.00 16800.00 16800.00 18394.00 19988.00
                             Euro 6         334.00 18793.06  2829.85 13450.00 16445.75 18333.00 21356.25 33900.00
                    Diesel   -               14.00 19012.00  2408.03 15500.00 16825.00 19099.50 20834.25 22900.00
                             Euro 5           1.00 16800.00      nan 16800.00 16800.00 16800.00 16800.00 16800.00
                             Euro 6         294.00 17762.28  2063.19 14900.00 16305.25 16890.00 18900.00 23700.00
               2.00 Benzine  -               57.00 17858.07  2525.66 14500.00 15450.00 17900.00 20490.00 23490.00
                             Euro 5           1.00 10999.00      nan 10999.00 10999.00 10999.00 10999.00 10999.00
                             Euro 6         213.00 16718.39  2172.89 12490.00 15400.00 15850.00 18300.00 22150.00
                    Diesel   -               29.00 15002.21  1467.29 11990.00 14391.00 14900.00 15680.00 18900.00
                             Euro 6         132.00 16343.38  1851.92 11600.00 15000.00 15900.00 17675.00 22500.00
               3.00 Benzine  -               58.00 14752.71  2108.49  9950.00 12982.50 14225.00 16712.50 18690.00
                             Euro 4           1.00 12990.00      nan 12990.00 12990.00 12990.00 12990.00 12990.00
                             Euro 6         260.00 14495.55  2027.21  8999.00 13150.00 13925.00 15960.00 18900.00
                    Diesel   -               35.00 14612.09  1958.52 11100.00 13485.00 13990.00 15742.00 18500.00
                             Euro 5           4.00 14210.00  1478.31 12950.00 12980.00 13995.00 15225.00 15900.00
                             Euro 6         271.00 14421.82  1699.50 10000.00 13400.00 14300.00 15090.00 19900.00
Audi A2        1.00 Diesel   Euro 6           1.00 28200.00      nan 28200.00 28200.00 28200.00 28200.00 28200.00
Audi A3        0.00 Benzine  -              104.00 25614.67  3902.89 19750.00 23895.00 24925.00 25650.00 41495.00
                             Euro 6         398.00 24575.96  3290.49 18820.00 23058.00 24442.00 25529.25 63900.00
                    Diesel   -              114.00 25228.12  1824.57 15500.00 24300.00 25435.00 26499.75 27400.00
                             Euro 6         202.00 24936.32  1953.76 20000.00 23900.00 24800.00 26297.50 32200.00
                    LPG/CNG  -                2.00 24300.00     0.00 24300.00 24300.00 24300.00 24300.00 24300.00
                             Euro 6           7.00 29361.14  3573.37 26300.00 26895.00 26900.00 31900.00 34738.00
               1.00 Benzine  -               33.00 22359.30  3664.60 18420.00 20550.00 21490.00 22990.00 39959.00
                             Euro 6         119.00 25687.66 11646.91 10893.00 20750.00 21490.00 23460.00 74600.00
                    Diesel   -               31.00 21614.35  1609.74 18700.00 20847.50 21500.00 22475.00 26900.00
                             Euro 5           1.00 20500.00      nan 20500.00 20500.00 20500.00 20500.00 20500.00
                             Euro 6         590.00 21844.55  2194.10  5555.00 20841.25 21500.00 22700.00 38000.00
                    Electric Euro 6           3.00 3744

In [97]:
df["Emission_Class"].replace("-", np.nan, inplace=True)

In [98]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [99]:
fill(df, "age", "Fuel", "Emission_Class", "ffill")

Number of NaN :  0
------------------
Euro 6    15770
Euro 5       99
Euro 4       50
Name: Emission_Class, dtype: int64


### Gears

In [100]:
df["Gears"].value_counts(dropna=False)

6.00     5822
nan      4712
5.00     3239
7.00     1908
8.00      224
9.00        6
4.00        2
3.00        2
1.00        2
50.00       1
2.00        1
Name: Gears, dtype: int64

In [101]:
df["Gears"].fillna("-", inplace=True)

In [102]:
df["Gears"].value_counts(dropna=False)

6.0     5822
-       4712
5.0     3239
7.0     1908
8.0      224
9.0        6
4.0        2
3.0        2
1.0        2
50.0       1
2.0        1
Name: Gears, dtype: int64

In [103]:
df.groupby(["make_model", "body_type", "Gears"]).price.describe()

count     mean      std      min      25%      50%      75%      max
make_model     body_type     Gears                                                                       
Audi A1        Compact       5.0    280.00 16392.20  3097.81  9950.00 13996.75 15900.00 16950.00 25256.00
                             6.0     83.00 20574.45  2252.10 12550.00 19488.00 20885.00 21987.50 28860.00
                             7.0    202.00 22088.50  3961.89 13990.00 18960.00 21845.00 24427.50 31990.00
                             8.0      1.00 16880.00      nan 16880.00 16880.00 16880.00 16880.00 16880.00
                             -      473.00 19368.94  4879.64 10490.00 15740.00 17900.00 22490.00 29197.00
               Coupe         5.0      1.00 13950.00      nan 13950.00 13950.00 13950.00 13950.00 13950.00
                             -        1.00 15900.00      nan 15900.00 15900.00 15900.00 15900.00 15900.00
               Sedans        5.0    527.00 15559.03  2538.30  8999.00 13890.00 15500.00 16700.00 22900.00
                             6.0    101.00 21510.11  3342.36 13000.00 20690.00 21990.00 22700.00 37900.00
                             7.0    451.00 21204.82  4311.40 11000.00 17925.00 20290.00 24925.00 35900.00
                             8.0      1.00 20500.00      nan 20500.00 20500.00 20500.00 20500.00 20500.00
                             -      471.00 19077.81  4376.97 11180.00 15787.00 17900.00 21960.00 29137.00
               Station wagon 5.0     10.00 15574.00  1198.62 12950.00 15000.00 15850.00 16432.50 16880.00
                             6.0      1.00 13979.00      nan 13979.00 13979.00 13979.00 13979.00 13979.00
                             7.0      3.00 23193.00  5069.98 19640.00 20290.00 20940.00 24969.50 28999.00
                             -        7.00 19248.71  4997.76 13999.00 16156.00 17440.00 21050.00 28890.00
               Van           6.0      1.00 29000.00      nan 29000.00 29000.00 29000.00 29000.00 29000.00
Audi A2        Off-Road      -        1.00 28200.00      nan 28200.00 28200.00 28200.00 28200.00 28200.00
Audi A3        Compact       5.0      1.00 18900.00      nan 18900.00 18900.00 18900.00 18900.00 18900.00
                             6.0     92.00 16262.80  3252.64 12900.00 13500.00 14970.00 18866.25 25272.00
                             7.0     70.00 22807.60  8991.81 13950.00 17557.75 20655.00 23412.50 67600.00
                             -       19.00 18465.47  2776.12 14530.00 16725.00 17894.00 19725.00 24900.00
               Convertible   6.0      2.00 25450.00  7707.46 20000.00 22725.00 25450.00 28175.00 30900.00
                             7.0      6.00 28723.83 15442.01 10893.00 21175.00 25750.00 32087.50 56100.00
               Coupe         7.0      4.00 17745.00   962.65 16990.00 16990.00 17495.00 18250.00 19000.00
               Sedans        5.0      1.00 12949.00      nan 12949.00 12949.00 12949.00 12949.00 12949.00
                             6.0   1053.00 20104.68  3631.93 10550.00 17490.00 19999.00 22590.00 38450.00
                             7.0    990.00 22860.01  5820.49 13000.00 19900.00 22425.00 24900.00 74600.00
                             -      577.00 20142.99  3950.12 10950.00 16500.00 20390.00 22999.00 28300.00
               Station wagon 6.0    111.00 20119.73  3513.72  5555.00 18455.00 19500.00 23100.00 25490.00
                             7.0     41.00 22568.15  7555.11 15700.00 18500.00 21500.00 24425.00 63900.00
                             -      130.00 20572.34  4252.86 11790.00 16992.25 21854.00 23998.25 27400.00
Opel Astra     Compact       5.0     33.00 16137.00  2258.76 10791.00 13899.00 16500.00 17490.00 19950.00
                             6.0    135.00 15837.93  6362.68  8450.00 12430.50 14300.00 16945.00 39433.00
                             -       17.00 19195.00  7899.98  9490.00 13995.00 16499.00 23620.00 33597.00
               Coupe         6.0      1.00 14490.00      nan 14490.00 14490.00 14490.00 14490.00 14490.00
         

In [104]:
df["Gears"].replace([1,2,3,4,9,50,"-"], [5,5,6,6,6,5,np.nan], inplace=True)

In [105]:
df["Gears"].value_counts(dropna=False)

6.00    5832
nan     4712
5.00    3243
7.00    1908
8.00     224
Name: Gears, dtype: int64

In [106]:
fill(df, "make_model", "body_type", "Gears", "mode")

Number of NaN :  0
------------------
6.00    8420
5.00    5367
7.00    1908
8.00     224
Name: Gears, dtype: int64


In [107]:
#df["Gears"] = df["Gears"].fillna(df.groupby("make_model")["Gears"].transform("median"))

### hp_kW

In [108]:
df["hp_kW"].value_counts(dropna=False)

85.00     2542
66.00     2122
81.00     1402
100.00    1308
110.00    1112
70.00      888
125.00     707
51.00      695
55.00      569
118.00     516
92.00      466
121.00     392
147.00     380
77.00      345
56.00      286
54.00      276
103.00     253
87.00      232
165.00     194
88.00      177
60.00      160
162.00      98
nan         88
74.00       81
96.00       72
71.00       59
101.00      47
67.00       40
154.00      39
122.00      35
119.00      30
164.00      27
135.00      24
82.00       22
52.00       22
1.00        20
78.00       20
294.00      18
146.00      18
141.00      16
57.00       10
104.00       8
120.00       8
112.00       7
191.00       7
117.00       6
155.00       6
184.00       5
65.00        4
90.00        4
76.00        4
98.00        3
168.00       3
93.00        3
149.00       3
80.00        3
140.00       2
270.00       2
86.00        2
40.00        2
150.00       2
89.00        2
53.00        2
63.00        2
143.00       2
127.00       2
167.00    

In [109]:
df["hp_kW"].fillna("-", inplace=True)

In [110]:
df.groupby(["make_model", "body_type","hp_kW"]).price.describe()

count     mean      std      min      25%      50%      75%      max
make_model     body_type     hp_kW                                                                       
Audi A1        Compact       60.0    58.00 15189.83  1663.61 10900.00 14390.00 15774.50 16345.00 16978.00
                             66.0   162.00 15398.21  1934.30 10490.00 14042.50 15465.00 16445.00 23700.00
                             70.0   332.00 17983.26  3315.81  9950.00 15480.00 17447.00 21190.00 28990.00
                             71.0    32.00 20831.59  2410.84 15890.00 18937.50 21425.00 22462.25 25256.00
                             85.0   330.00 22604.10  4572.57 11100.00 19700.00 22497.00 26980.00 31990.00
                             86.0     1.00 14295.00      nan 14295.00 14295.00 14295.00 14295.00 14295.00
                             92.0    90.00 18029.06  3027.43 12550.00 15850.00 16935.00 20624.25 28880.00
                             93.0     2.00 21447.50   774.28 20900.00 21173.75 21447.50 21721.25 21995.00
                             110.0   20.00 23620.45  3777.34 15490.00 21299.50 23325.00 26932.50 28980.00
                             141.0    2.00 22495.00   841.46 21900.00 22197.50 22495.00 22792.50 23090.00
                             147.0    8.00 28797.12   355.36 28200.00 28535.00 28900.00 29042.50 29181.00
                             -        2.00 19125.00  5126.52 15500.00 17312.50 19125.00 20937.50 22750.00
               Coupe         70.0     1.00 13950.00      nan 13950.00 13950.00 13950.00 13950.00 13950.00
                             85.0     1.00 15900.00      nan 15900.00 15900.00 15900.00 15900.00 15900.00
               Sedans        60.0    95.00 15218.17  1624.27 11900.00 14099.50 15300.00 15984.00 22390.00
                             66.0   296.00 15922.80  2331.61 10800.00 14200.00 15950.00 16896.25 23500.00
                             67.0    12.00 15432.42  2778.19 11990.00 13790.00 14837.00 17118.75 21500.00
                             70.0   455.00 17282.07  3210.46  8999.00 14845.00 16810.00 19990.00 25900.00
                             71.0    24.00 19177.25  2539.50 14390.00 17967.50 19990.00 20922.50 22789.00
                             85.0   555.00 21609.52  4822.01 10000.00 16900.00 22000.00 25900.00 35900.00
                             87.0     4.00 14865.00   750.00 14490.00 14490.00 14490.00 14865.00 15990.00
                             92.0    79.00 19078.97  2790.21 13000.00 17515.00 18900.00 20942.50 27650.00
                             93.0     1.00 17900.00      nan 17900.00 17900.00 17900.00 17900.00 17900.00
                             110.0    9.00 22591.33  4188.63 16999.00 21460.00 22440.00 22570.00 29000.00
                             112.0    1.00 18399.00      nan 18399.00 18399.00 18399.00 18399.00 18399.00
                             140.0    1.00 23490.00      nan 23490.00 23490.00 23490.00 23490.00 23490.00
                             141.0   14.00 21232.00  2597.47 14500.00 20542.50 22056.50 22960.00 23750.00
                             147.0    5.00 30729.60  4008.78 28880.00 28880.00 28990.00 28998.00 37900.00
               Station wagon 60.0     3.00 15106.67  1890.88 12950.00 14420.00 15890.00 16185.00 16480.00
                             66.0     7.00 16018.43  1779.76 13999.00 15250.00 15850.00 16070.00 19640.00
                             70.0     3.00 17420.00  3283.47 14440.00 15660.00 16880.00 18910.00 20940.00
                             85.0     4.00 22702.75  7207.56 16422.00 16480.50 22695.00 28917.25 28999.00
                             92.0     3.00 17623.00  3738.86 13979.00 15709.50 17440.00 19445.00 21450.00
                             110.0    1.00 20650.00      nan 20650.00 20650.00 20650.00 20650.00 20650.00
               Van           147.0    1.00 29000.00      nan 29000.00 29000.00 29000.00 29000.00 29000.00
Audi A2        Off-Road      85.0     1.00 28200.00      nan 28200.00 28200.00 28200.00 28200.00 28200.00
Audi A3  

In [111]:
df["hp_kW"].replace("-", np.nan, inplace=True)

In [112]:
fill(df, "make_model", "body_type", "hp_kW", "median")

Number of NaN :  0
------------------
85.00     2543
66.00     2124
81.00     1404
100.00    1308
110.00    1112
70.00      890
125.00     707
51.00      695
55.00      570
118.00     550
92.00      466
121.00     396
147.00     380
77.00      353
56.00      314
54.00      276
103.00     259
87.00      232
165.00     194
88.00      177
60.00      160
162.00      98
74.00       81
96.00       72
71.00       59
101.00      47
67.00       40
154.00      39
122.00      35
119.00      30
164.00      27
135.00      24
82.00       22
52.00       22
1.00        20
78.00       20
146.00      18
294.00      18
141.00      16
57.00       10
104.00       8
120.00       8
191.00       7
112.00       7
117.00       6
155.00       6
184.00       5
76.00        4
90.00        4
65.00        4
98.00        3
168.00       3
149.00       3
80.00        3
93.00        3
150.00       2
143.00       2
167.00       2
140.00       2
127.00       2
228.00       2
89.00        2
270.00       2
40.00        2
86

## Displacement_cc

In [113]:
df["Displacement_cc"].value_counts(dropna=False)

1598.00     4761
999.00      2438
1398.00     1314
1399.00      749
1229.00      677
1956.00      670
1461.00      595
1490.00      559
nan          496
1422.00      467
1197.00      353
898.00       351
1395.00      320
1968.00      301
1149.00      288
1618.00      212
1798.00      210
1498.00      196
1600.00      130
1248.00      110
1997.00      103
1364.00      102
1400.00       90
998.00        72
1500.00       50
2000.00       46
1000.00       40
1.00          36
1998.00       25
2480.00       20
1984.00       18
1200.00       18
899.00        11
1397.00       11
160.00         6
1499.00        5
929.00         5
900.00         4
997.00         4
1596.00        4
139.00         4
1396.00        3
1599.00        3
1199.00        3
995.00         2
1495.00        2
2.00           2
1300.00        2
1589.00        2
16000.00       1
1568.00        1
1333.00        1
15898.00       1
1686.00        1
1100.00        1
1369.00        1
1239.00        1
996.00         1
1533.00       

In [114]:
df["Displacement_cc"].fillna("-", inplace=True)

In [115]:
df.groupby(["make_model", "body_type","Displacement_cc"]).price.describe()

count     mean      std      min      25%      50%      75%      max
make_model     body_type     Displacement_cc                                                                       
Audi A1        Compact       929.0              5.00 16364.00  1504.82 14980.00 15480.00 16240.00 16240.00 18880.00
                             995.0              2.00 21990.00  1414.21 20990.00 21490.00 21990.00 22490.00 22990.00
                             999.0            663.00 20294.10  4570.80  9950.00 16480.00 20380.00 22820.00 31990.00
                             1000.0             7.00 19498.43  5396.20 13500.00 16024.50 16800.00 22695.00 28750.00
                             1395.0            98.00 18246.62  3084.08 12550.00 15850.00 17835.00 20900.00 28880.00
                             1422.0           158.00 15396.42  1943.65 10490.00 14035.00 15465.00 16445.00 23700.00
                             1498.0            14.00 24982.14  3186.06 20582.00 22453.00 24382.50 28380.00 28980.00
                             1596.0             1.00 15585.00      nan 15585.00 15585.00 15585.00 15585.00 15585.00
                             1598.0            68.00 16568.31  3059.45 11100.00 14271.25 15900.00 18263.00 23500.00
                             1600.0             1.00 16800.00      nan 16800.00 16800.00 16800.00 16800.00 16800.00
                             1798.0             2.00 22495.00   841.46 21900.00 22197.50 22495.00 22792.50 23090.00
                             1984.0             8.00 28797.12   355.36 28200.00 28535.00 28900.00 29042.50 29181.00
                             -                 12.00 17960.83  5247.78 13700.00 14467.50 15600.00 18700.00 28490.00
               Coupe         999.0              1.00 13950.00      nan 13950.00 13950.00 13950.00 13950.00 13950.00
                             1598.0             1.00 15900.00      nan 15900.00 15900.00 15900.00 15900.00 15900.00
               Sedans        998.0              2.00 20990.00     0.00 20990.00 20990.00 20990.00 20990.00 20990.00
                             999.0            882.00 19883.03  4867.72  8999.00 15842.50 19895.00 22990.00 35900.00
                             1000.0             7.00 26414.29  3161.01 21750.00 23800.00 28690.00 28935.00 28990.00
                             1197.0             1.00 15980.00      nan 15980.00 15980.00 15980.00 15980.00 15980.00
                             1395.0            85.00 18973.65  2474.70 13000.00 17500.00 18900.00 20990.00 22991.00
                             1422.0           302.00 15917.01  2361.31 10800.00 13992.50 15955.00 16900.00 23500.00
                             1498.0             2.00 28950.00    70.71 28900.00 28925.00 28950.00 28975.00 29000.00
                             1596.0             1.00 17700.00      nan 17700.00 17700.00 17700.00 17700.00 17700.00
                             1598.0           233.00 16802.60  2465.63 10000.00 15090.00 16400.00 18850.00 22930.00
                             1600.0             1.00 13290.00      nan 13290.00 13290.00 13290.00 13290.00 13290.00
                             1798.0            14.00 21232.00  2597.47 14500.00 20542.50 22056.50 22960.00 23750.00
                             1984.0             5.00 30729.60  4008.78 28880.00 28880.00 28990.00 28998.00 37900.00
                             -                 16.00 17966.88  3330.74 12600.00 15487.50 17350.00 20900.00 23490.00
               Station wagon 999.0              7.00 18067.14  5375.83 12950.00 15165.00 16480.00 18910.00 28890.00
                             1395.0             4.00 18379.75  3407.35 13979.00 16574.75 19045.00 20850.00 21450.00
                             1422.0             7.00 16018.43  1779.76 13999.00 15250.00 15850.00 16070.00 19640.00
                             1598.0             2.00 16461.00    55.15 16422.00 16441.50 16461.00 16480.50 16500.00
                             -                  1.00 28999.00      nan 28999.00 28999.00 28999.00 28999

In [116]:
df["Displacement_cc"].replace("-", np.nan, inplace=True)

In [117]:
fill(df, "make_model", "body_type", "Displacement_cc", "median")

Number of NaN :  0
------------------
1598.00     5042
999.00      2466
1398.00     1334
1399.00      768
1229.00      677
1956.00      670
1461.00      596
1490.00      559
1422.00      467
1197.00      387
898.00       351
1395.00      321
1968.00      301
1149.00      288
1618.00      212
1798.00      210
1498.00      196
1600.00      130
1248.00      110
1997.00      103
1364.00      102
1400.00       90
1200.00       74
998.00        72
1357.00       53
1500.00       50
2000.00       46
1000.00       40
1.00          36
1998.00       25
2480.00       20
1984.00       18
899.00        11
1397.00       11
160.00         6
929.00         5
1499.00        5
997.00         4
900.00         4
1596.00        4
139.00         4
1396.00        3
1599.00        3
1199.00        3
1608.00        2
995.00         2
1300.00        2
1589.00        2
1495.00        2
2.00           2
996.00         1
1239.00        1
1369.00        1
1686.00        1
1100.00        1
15898.00       1
1533.00   

## Weight_kg

In [118]:
df["Weight_kg"].value_counts(dropna=False)

nan        6974
1163.00     574
1360.00     356
1165.00     301
1335.00     242
1135.00     213
1199.00     205
1734.00     170
1180.00     168
1503.00     165
1350.00     155
1355.00     135
1260.00     127
1275.00     112
1278.00     110
1487.00     109
1425.00     109
1255.00     108
1200.00     107
1273.00     103
1522.00     103
1280.00     102
1403.00      91
1120.00      90
1659.00      89
1195.00      89
1701.00      87
1250.00      84
1441.00      82
1285.00      80
1308.00      80
1110.00      75
1613.00      75
1279.00      72
1364.00      70
1345.00      67
1733.00      65
1141.00      64
1685.00      64
1325.00      64
1071.00      64
1230.00      63
1845.00      56
1090.00      54
1052.00      53
1664.00      52
1154.00      52
1513.00      51
1065.00      50
1237.00      49
1205.00      46
1440.00      46
1088.00      46
1119.00      46
1265.00      45
1395.00      44
1585.00      43
1209.00      43
1666.00      43
1365.00      42
1162.00      42
1134.00      42
1248.00 

In [119]:
df["Weight_kg"].fillna("-", inplace=True)

In [120]:
df.groupby(["make_model", "body_type","Weight_kg"]).price.describe()

count     mean      std      min      25%      50%      75%      max
make_model     body_type     Weight_kg                                                                      
Audi A1        Compact       102.0       1.00 19229.00      nan 19229.00 19229.00 19229.00 19229.00 19229.00
                             1010.0      2.00 15450.00   707.11 14950.00 15200.00 15450.00 15700.00 15950.00
                             1035.0      6.00 16796.67  2617.87 14390.00 15892.50 15900.00 16575.00 21900.00
                             1040.0      2.00 20424.50  2933.79 18350.00 19387.25 20424.50 21461.75 22499.00
                             1065.0     36.00 20971.78  1982.55 15500.00 18987.50 21690.00 22400.00 23550.00
                             1090.0      9.00 21059.22  2945.23 15998.00 18990.00 22450.00 22800.00 25256.00
                             1100.0      2.00 27525.00  1732.41 26300.00 26912.50 27525.00 28137.50 28750.00
                             1105.0     12.00 13629.17   228.09 13400.00 13400.00 13625.00 13825.00 13900.00
                             1110.0     22.00 14322.00  1738.10  9950.00 13116.75 14559.50 15430.00 16879.00
                             1115.0      3.00 20833.33  1950.85 18850.00 19875.00 20900.00 21825.00 22750.00
                             1125.0      1.00 13945.00      nan 13945.00 13945.00 13945.00 13945.00 13945.00
                             1133.0      3.00 21223.33   251.66 20990.00 21090.00 21190.00 21340.00 21490.00
                             1134.0      6.00 16258.33   988.14 14900.00 15637.50 16300.00 16850.00 17600.00
                             1135.0     53.00 16554.68  2170.54 12980.00 15444.00 16490.00 16900.00 22690.00
                             1138.0      1.00 22140.00      nan 22140.00 22140.00 22140.00 22140.00 22140.00
                             1140.0      1.00 21940.00      nan 21940.00 21940.00 21940.00 21940.00 21940.00
                             1147.0      1.00 22990.00      nan 22990.00 22990.00 22990.00 22990.00 22990.00
                             1150.0      1.00 22980.00      nan 22980.00 22980.00 22980.00 22980.00 22980.00
                             1155.0      5.00 18533.00  3476.37 13480.00 16430.00 19990.00 21275.00 21490.00
                             1165.0     77.00 20264.26  2560.32 12900.00 18333.00 21390.00 22400.00 23777.00
                             1180.0     86.00 20753.62  2638.05 13475.00 19388.00 20617.50 22290.00 29181.00
                             1195.0     40.00 15930.73  1308.22 11290.00 15740.00 16430.00 16445.00 18490.00
                             1200.0     65.00 26139.55  2769.60 18730.00 25490.00 27380.00 27980.00 28980.00
                             1205.0      5.00 15098.80  1223.64 13750.00 13980.00 15490.00 15585.00 16689.00
                             1215.0      9.00 20182.56  2365.73 16879.00 17970.00 20669.00 21900.00 23450.00
                             1220.0      2.00 18090.00  3733.52 15450.00 16770.00 18090.00 19410.00 20730.00
                             1225.0      4.00 15887.75  1550.41 14391.00 14982.75 15580.00 16485.00 18000.00
                             1230.0      3.00 23250.00  2655.65 20500.00 21975.00 23450.00 24625.00 25800.00
                             1235.0      2.00 16590.00   141.42 16490.00 16540.00 16590.00 16640.00 16690.00
                             1240.0      7.00 25600.71  3398.83 20980.00 22730.00 26450.00 28667.50 28980.00
                             1250.0     17.00 16218.59  3691.28 11100.00 14400.00 15450.00 16800.00 28490.00
                             1275.0      5.00 20453.80  4993.47 15889.00 15930.00 20700.00 21770.00 27980.00
                             1280.0      1.00 25490.00      nan 25490.00 25490.00 25490.00 25490.00 25490.00
                             1285.0      3.00 15516.67  1590.99 13680.00 15040.00 16400.00 16435.00 16470.00
                             1335.0      3.00 28832.33   237.56 28560.00 28750.00 28940.00 28968.50 28997.00
       

In [121]:
df["Weight_kg"].replace("-", np.nan, inplace=True)

In [122]:
fill(df, "make_model", "body_type", "Weight_kg", "median")

Number of NaN :  0
------------------
1163.00    1547
1350.00    1147
1613.00     931
1195.00     712
1180.00     694
1393.00     675
1308.00     667
1503.00     554
1335.00     411
1691.00     377
1360.00     356
1162.00     344
1090.00     306
1165.00     301
1135.00     213
1199.00     205
1734.00     177
1325.00     136
1355.00     135
1260.00     127
1280.00     127
1175.00     118
1275.00     112
1487.00     110
1278.00     110
1425.00     109
1255.00     108
1200.00     107
1273.00     104
1522.00     103
1403.00      91
1120.00      90
1659.00      89
1701.00      87
1250.00      85
1441.00      82
1285.00      80
1110.00      75
1279.00      72
1364.00      70
1345.00      67
1733.00      65
1141.00      64
1685.00      64
1071.00      64
1230.00      63
1845.00      56
1205.00      53
1052.00      53
1154.00      52
1664.00      52
1513.00      51
1065.00      50
1237.00      49
1088.00      46
1440.00      46
1119.00      46
1265.00      45
1395.00      44
1209.00      43
16

In [123]:
#df.drop("Weight_kg", axis=1, inplace=True)

## CO2 Emission

In [124]:
df["CO2_Emission"].value_counts(dropna=False)

nan       2436
120.00     740
99.00      545
97.00      537
104.00     501
10.00      477
103.00     445
114.00     382
124.00     372
107.00     362
108.00     362
119.00     361
106.00     349
128.00     329
126.00     282
85.00      275
118.00     270
110.00     266
127.00     257
117.00     254
111.00     237
113.00     235
109.00     234
139.00     224
140.00     218
129.00     213
135.00     202
105.00     202
9.00       198
130.00     180
123.00     178
150.00     174
143.00     171
11.00      171
95.00      161
116.00     157
141.00     156
98.00      151
133.00     145
136.00     145
137.00     133
125.00     132
134.00     130
145.00     126
149.00     117
153.00     113
147.00     109
101.00     105
13.00      100
115.00      86
1.00        84
121.00      82
138.00      75
93.00       66
14.00       59
168.00      58
148.00      48
131.00      48
154.00      40
144.00      40
94.00       37
146.00      36
100.00      36
90.00       35
151.00      34
91.00       31
186.00    

In [125]:
df["CO2_Emission"].fillna("-", inplace=True)

In [126]:
df.groupby(["make_model", "body_type","CO2_Emission"]).price.describe()

count     mean      std      min      25%      50%      75%      max
make_model     body_type     CO2_Emission                                                                            
Audi A1        Compact       1.0                  1.00 20881.00      nan 20881.00 20881.00 20881.00 20881.00 20881.00
                             9.0                  1.00 12479.00      nan 12479.00 12479.00 12479.00 12479.00 12479.00
                             10.0               147.00 16946.87  2850.06  9950.00 15765.00 16450.00 18333.00 29150.00
                             11.0                25.00 19960.36  2676.26 14930.00 17970.00 19990.00 21460.00 28980.00
                             14.0                 2.00 28750.00   268.70 28560.00 28655.00 28750.00 28845.00 28940.00
                             90.0                 1.00 12880.00      nan 12880.00 12880.00 12880.00 12880.00 12880.00
                             91.0                 5.00 13038.00  1108.93 11800.00 11850.00 13800.00 13800.00 13940.00
                             94.0                 5.00 13960.00  1232.07 12900.00 12900.00 13900.00 14200.00 15900.00
                             97.0               113.00 15449.59  2445.75 10900.00 13770.00 15400.00 16700.00 23500.00
                             98.0                33.00 16574.36  2637.09 13999.00 15290.00 15900.00 16800.00 25256.00
                             99.0                19.00 16043.95  2257.40 10900.00 14585.00 16900.00 17925.00 18880.00
                             100.0                4.00 16127.50  1583.40 14220.00 15247.50 16195.00 17075.00 17900.00
                             103.0               30.00 21233.03  1794.55 18350.00 19225.00 22189.50 22448.50 23550.00
                             104.0              126.00 17411.41  3149.37 11445.00 14990.00 16480.00 20977.25 23700.00
                             105.0               14.00 22234.86   812.26 20840.00 21746.75 22490.00 22861.75 22990.00
                             106.0               43.00 19831.79  3335.79 13685.00 16410.00 21770.00 22490.00 22990.00
                             107.0               13.00 16527.69  1620.41 14470.00 15830.00 16430.00 16430.00 19890.00
                             108.0               42.00 23587.10  3375.69 15970.00 20990.00 23565.00 25942.50 29190.00
                             109.0               34.00 17029.32  2514.31 12750.00 15765.00 16455.00 16980.00 23829.00
                             110.0               51.00 26340.37  2434.74 19990.00 24825.00 26980.00 28290.00 29197.00
                             111.0              138.00 23765.28  3477.92 18288.00 20888.75 22785.00 27380.00 28990.00
                             113.0                5.00 20388.00  2796.19 16480.00 18620.00 21490.00 21900.00 23450.00
                             114.0                1.00 15490.00      nan 15490.00 15490.00 15490.00 15490.00 15490.00
                             115.0                7.00 24067.86  6950.65 13480.00 20450.00 28380.00 28420.00 28875.00
                             116.0               20.00 23580.95  2813.10 17900.00 22175.00 22865.00 23972.75 28999.00
                             117.0                8.00 19094.88  2942.95 14700.00 16732.50 19779.50 21080.00 22790.00
                             118.0                2.00 13615.00    91.92 13550.00 13582.50 13615.00 13647.50 13680.00
                             119.0                1.00 12550.00      nan 12550.00 12550.00 12550.00 12550.00 12550.00
                             120.0               27.00 15850.00     0.00 15850.00 15850.00 15850.00 15850.00 15850.00
                             121.0                2.00 18710.00  3224.41 16430.00 17570.00 18710.00 19850.00 20990.00
                             123.0               17.00 19308.65  3816.07 13475.00 16879.00 18880.00 21950.00 28880.00
                             126.0                1.00 19999.00      nan 19999.00 19999.00 19999.00 19999.00 19999.00
                             127.0   

In [127]:
df["CO2_Emission"].replace("-", np.nan, inplace=True)

In [128]:
fill(df, "make_model", "body_type", "CO2_Emission", "median")

Number of NaN :  0
------------------
120.00    836
104.00    679
106.00    674
114.00    556
99.00     546
97.00     539
139.00    523
124.00    516
10.00     477
117.00    460
103.00    445
123.00    410
98.00     366
107.00    362
108.00    362
119.00    361
129.00    333
133.00    330
128.00    329
118.00    314
110.00    289
126.00    284
85.00     275
127.00    257
111.00    237
113.00    235
109.00    234
140.00    221
105.00    202
135.00    202
9.00      198
130.00    189
150.00    174
11.00     171
143.00    171
95.00     161
116.00    157
141.00    156
118.50    147
136.00    145
137.00    135
125.00    133
134.00    130
145.00    126
149.00    117
153.00    113
147.00    109
101.00    105
13.00     100
115.00     86
1.00       84
121.00     82
138.00     75
93.00      66
14.00      59
168.00     58
90.00      54
131.00     48
148.00     48
144.00     40
154.00     40
94.00      37
146.00     36
100.00     36
151.00     34
91.00      31
186.00     31
15.00      28
187.00    

### Comfort & Convenience

In [129]:
df["Comfort&Convenience"].value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                             920
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                              216
Air conditioning,Electrical side mirrors,Power windows                                                                                                                                                                                                          

In [130]:
fill(df, "make_model", "body_type", "Comfort&Convenience", "mode")

Number of NaN :  0
------------------
Air conditioning,Electrical side mirrors,Hill Holder,Power windows                                                                                                                                                                                                                                                                                                                                                                                                                             388
Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system                                                                                                                                             

In [131]:
df["Comfort&Convenience"].str.get_dummies(sep = ",").add_prefix("cc_")

,cc_Air conditioning,cc_Air suspension,cc_Armrest,cc_Automatic climate control,cc_Auxiliary heating,cc_Cruise control,cc_Electric Starter,cc_Electric tailgate,cc_Electrical side mirrors,cc_Electrically adjustable seats,cc_Electrically heated windshield,cc_Heads-up display,cc_Heated steering wheel,cc_Hill Holder,cc_Keyless central door lock,cc_Leather seats,cc_Leather steering wheel,cc_Light sensor,cc_Lumbar support,cc_Massage seats,cc_Multi-function steering wheel,cc_Navigation system,cc_Panorama roof,cc_Park Distance Control,cc_Parking assist system camera,cc_Parking assist system self-steering,cc_Parking assist system sensors front,cc_Parking assist system sensors rear,cc_Power windows,cc_Rain sensor,cc_Seat heating,cc_Seat ventilation,cc_Split rear seats,cc_Start-stop system,cc_Sunroof,cc_Tinted windows,cc_Wind deflector,cc_Windshield
0,1,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0
2,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,0
3,0,1,1,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0
4,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,1,0,0,1,0,1,0,0,1,1,0,0,0,1,1,0,1,1,0,1,1,1,0,1,0,0,1,1,1,1,0,0,1,1,0,1,0,0
15915,1,0,0,1,0,1,0,1,1,0,1,1,1,1,1,0,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0
15916,1,0,1,1,0,1,0,1,1,0,1,1,1,0,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,1,0,0,0
15917,1,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [132]:
df = df.join(df["Comfort&Convenience"].str.get_dummies(sep = ",").add_prefix("cc_"))

In [133]:
df.drop("Comfort&Convenience", axis=1, inplace=True)

### Entertainment & Media

In [134]:
df["Entertainment&Media"].value_counts(dropna=False)

NaN                                                                                                               1374
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                                        1282
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                                     982
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                                           783
On-board computer,Radio                                                                                            487
Radio                                                                                                              477
Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB                                               465
On-board computer                                                                                                  461
Bluetooth,CD player,Hands-free equipment,MP3,On-

In [135]:
fill(df, "make_model", "body_type", "Entertainment&Media", "mode")

Number of NaN :  0
------------------
Bluetooth,Hands-free equipment,On-board computer,Radio,USB                                                        1738
Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB                                                    1134
Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB                                          1010
On-board computer                                                                                                  615
Radio                                                                                                              558
Bluetooth,Hands-free equipment,On-board computer,Radio                                                             515
On-board computer,Radio                                                                                            487
Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB                                               466
Bluetooth,

In [136]:
df = df.join(df["Entertainment&Media"].str.get_dummies(sep = ",").add_prefix("em_"))

In [137]:
df.drop("Entertainment&Media", axis=1, inplace=True)

### Extras

In [138]:
df["Extras"].value_counts(dropna=False)

Alloy wheels                                                                                                                                   3245
NaN                                                                                                                                            2962
Alloy wheels,Touch screen                                                                                                                       697
Alloy wheels,Voice Control                                                                                                                      577
Alloy wheels,Touch screen,Voice Control                                                                                                         541
Alloy wheels,Roof rack                                                                                                                          385
Alloy wheels,Sport seats                                                                                        

In [139]:
fill(df, "make_model", "body_type", "Extras", "mode")

Number of NaN :  0
------------------
Alloy wheels                                                                                                                                   5786
Alloy wheels,Touch screen                                                                                                                       697
Roof rack                                                                                                                                       596
Alloy wheels,Voice Control                                                                                                                      582
Alloy wheels,Touch screen,Voice Control                                                                                                         544
Alloy wheels,Roof rack                                                                                                                          529
Alloy wheels,Sport seats                                                  

In [140]:
df = df.join(df["Extras"].str.get_dummies(sep = ",").add_prefix("ex_"))

In [141]:
df.drop("Extras", axis=1, inplace=True)

### Safety & Security

In [142]:
df["Safety&Security"].value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                982
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                       538
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                                  480
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Pas

In [143]:
fill(df, "make_model", "body_type", "Safety&Security", "mode")

Number of NaN :  0
------------------
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                       729
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                                                                  480
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control                            373
ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-sid

In [144]:
df = df.join(df["Safety&Security"].str.get_dummies(sep = ",").add_prefix("ss_"))

In [145]:
df.drop("Safety&Security", axis=1, inplace=True)

## cons_comb

In [146]:
df["cons_comb"].value_counts(dropna=False)

nan      2033
5.40      770
3.90      733
4.00      713
5.10      657
4.40      623
5.60      618
4.70      602
3.80      585
4.80      546
5.00      545
4.50      523
5.20      454
4.20      435
4.60      426
4.90      393
5.50      380
5.30      380
5.90      369
3.70      369
5.70      342
4.10      342
6.00      331
3.30      307
4.30      307
3.50      288
6.20      216
3.60      194
6.30      181
6.10      175
5.80      164
6.60      148
6.80      136
3.40      106
6.40       75
3.00       69
7.40       66
6.70       43
6.50       43
7.10       38
10.00      34
6.90       27
3.20       25
8.30       20
7.60       14
7.00       10
3.10        7
7.20        6
7.80        6
8.00        5
8.60        4
51.00       4
8.70        3
7.90        3
1.60        3
38.00       2
40.00       2
7.30        2
8.10        2
0.00        2
11.00       1
7.50        1
54.00       1
13.80       1
55.00       1
33.00       1
1.20        1
32.00       1
1.00        1
9.10        1
46.00       1
50.00 

In [147]:
cons_comb = (df["cons_country"] + df["cons_city"])/2

In [148]:
df["cons_comb"] = df["cons_comb"].fillna(cons_comb)

In [149]:
df["cons_comb"].value_counts(dropna=False)

nan      1925
5.40      770
3.90      733
4.00      713
5.10      657
4.40      623
5.60      618
4.70      607
3.80      585
4.80      546
5.00      545
4.50      524
5.20      454
4.20      435
4.60      426
4.90      393
5.30      380
5.50      380
3.70      369
5.90      369
5.70      342
4.10      342
6.00      331
4.30      307
3.30      307
3.50      288
6.20      216
3.60      194
6.30      181
6.10      175
5.80      165
6.60      148
6.80      136
3.40      125
6.40       75
3.00       69
7.40       66
6.50       43
6.70       43
7.10       38
3.60       37
10.00      34
6.90       27
3.20       25
8.30       20
3.65       20
5.15       19
7.60       14
7.00       10
3.10        7
7.20        6
7.80        6
8.00        5
51.00       4
8.60        4
3.95        3
1.60        3
8.70        3
7.90        3
40.00       2
8.10        2
38.00       2
5.45        2
7.30        2
0.00        2
1.20        1
11.00       1
7.50        1
13.80       1
9.10        1
54.00       1
32.00 

In [150]:
df["cons_comb"].fillna("-", inplace=True)

In [151]:
df.groupby(["make_model", "body_type","cons_comb"]).price.describe()

count     mean      std      min      25%      50%      75%      max
make_model     body_type     cons_comb                                                                               
Audi A1        Compact       3.0                  4.00 14370.25  4481.39 11100.00 11242.50 12840.50 15968.25 20700.00
                             3.4                  6.00 12755.00   900.75 11800.00 11987.50 12640.00 13570.00 13800.00
                             3.5                  3.00 14613.00  1848.10 12479.00 14079.50 15680.00 15680.00 15680.00
                             3.6                  5.00 13960.00  1232.07 12900.00 12900.00 13900.00 14200.00 15900.00
                             3.7                 50.00 15902.20  2960.32 12900.00 13900.00 15040.00 15967.50 23500.00
                             3.8                 18.00 16158.06  2265.76 10900.00 15240.00 16900.00 17927.50 18880.00
                             3.9                 41.00 15771.90  1567.31 10490.00 15780.00 16445.00 16680.00 16950.00
                             4.0                 79.00 16795.67  2620.87 12200.00 14990.00 15850.00 18583.50 23700.00
                             4.1                 38.00 16540.74  1754.46 12750.00 15870.00 16430.00 16869.75 22390.00
                             4.2                 87.00 15631.13  2420.35 10900.00 13970.00 15599.00 16822.50 25256.00
                             4.3                 17.00 16628.71  2467.67 13999.00 15290.00 15900.00 16900.00 22919.00
                             4.4                193.00 17618.92  3250.85  9950.00 15689.00 16950.00 19990.00 28750.00
                             4.5                 11.00 21482.91  1918.66 18400.00 19744.00 22444.00 23075.00 23250.00
                             4.6                 52.00 20729.96  2195.46 14470.00 19890.00 21490.00 22095.00 22990.00
                             4.7                 33.00 22102.67  1889.93 15490.00 21490.00 22400.00 22790.00 27980.00
                             4.8                 93.00 25162.55  3080.47 17330.00 22785.00 25750.00 27980.00 29197.00
                             4.9                176.00 23133.29  3723.04 13480.00 20288.00 22490.00 26980.00 28999.00
                             5.0                  9.00 21739.44  2364.49 17900.00 20582.00 21900.00 23450.00 24706.00
                             5.1                 50.00 18187.28  4445.45 12550.00 15850.00 15850.00 20759.75 28980.00
                             5.2                 18.00 19429.83  3737.66 13475.00 16881.75 19260.00 21835.00 28880.00
                             5.6                  2.00 19888.00     0.00 19888.00 19888.00 19888.00 19888.00 19888.00
                             5.8                  2.00 22495.00   841.46 21900.00 22197.50 22495.00 22792.50 23090.00
                             6.0                  8.00 28797.12   355.36 28200.00 28535.00 28900.00 29042.50 29181.00
                             13.8                 1.00 12990.00      nan 12990.00 12990.00 12990.00 12990.00 12990.00
                             -                   43.00 16765.79  4426.03 11480.00 13980.00 15290.00 19630.00 31990.00
               Coupe         4.2                  1.00 13950.00      nan 13950.00 13950.00 13950.00 13950.00 13950.00
                             -                    1.00 15900.00      nan 15900.00 15900.00 15900.00 15900.00 15900.00
               Sedans        3.0                  1.00 15450.00      nan 15450.00 15450.00 15450.00 15450.00 15450.00
                             3.4                 20.00 14829.40  1517.96 12699.00 13800.00 14599.50 16272.50 16900.00
                             3.5                 48.00 15679.12  1535.78 12890.00 15090.00 15090.00 15680.00 18900.00
                             3.6                 13.00 15590.00  2503.51 11440.00 14350.00 15000.00 17500.00 19900.00
                             3.7                140.00 15242.10  2130.60 10800.00 13890.00 14999.50 16500.00 20800.00
                             3.8     

In [152]:
df["cons_comb"].replace([0.0, 1.0, 1.2, 1.6, 10, 11, 13.8, 32.0, 33.0, 38.0, 40.0, 43.0, 46.0, 50.0, 51.0, 54.0, 55.0, "-"], np.nan, inplace=True)

In [153]:
df["cons_comb"].value_counts(dropna=False)

nan     1983
5.40     770
3.90     733
4.00     713
5.10     657
4.40     623
5.60     618
4.70     607
3.80     585
4.80     546
5.00     545
4.50     524
5.20     454
4.20     435
4.60     426
4.90     393
5.50     380
5.30     380
3.70     369
5.90     369
4.10     342
5.70     342
6.00     331
4.30     307
3.30     307
3.50     288
6.20     216
3.60     194
6.30     181
6.10     175
5.80     165
6.60     148
6.80     136
3.40     125
6.40      75
3.00      69
7.40      66
6.50      43
6.70      43
7.10      38
3.60      37
6.90      27
3.20      25
8.30      20
3.65      20
5.15      19
7.60      14
7.00      10
3.10       7
7.20       6
7.80       6
8.00       5
8.60       4
3.95       3
8.70       3
7.90       3
8.10       2
7.30       2
5.45       2
4.55       1
7.50       1
9.10       1
Name: cons_comb, dtype: int64

In [154]:
fill(df, "make_model", "body_type", "cons_comb", "median")

Number of NaN :  0
------------------
4.00    1020
5.60     965
5.40     900
5.00     810
4.70     766
3.90     733
4.50     731
4.40     668
5.10     657
3.80     585
4.80     580
4.20     579
4.90     535
5.30     530
5.20     454
4.60     426
5.50     389
3.70     369
5.90     369
4.10     362
5.70     342
6.00     331
4.30     307
3.30     307
3.50     288
6.20     219
3.60     194
6.30     181
6.10     175
5.80     165
6.60     148
6.80     136
3.40     125
6.40      75
3.00      69
7.40      66
6.70      43
6.50      43
7.10      38
3.60      37
6.90      27
3.20      25
8.30      20
3.65      20
3.45      19
5.15      19
7.60      14
7.00      10
3.10       7
7.20       6
7.80       6
8.00       5
8.60       4
8.70       3
3.95       3
7.90       3
8.10       2
7.30       2
5.45       2
4.55       2
7.50       1
4.30       1
9.10       1
Name: cons_comb, dtype: int64


## cons_country

In [155]:
df.drop("cons_country", axis = 1, inplace = True)

## cons_city 

In [156]:
df.drop("cons_city", axis = 1, inplace = True)

## End of this phase

In [157]:
df.shape

(15919, 119)

In [158]:
df.isnull().sum()/df.shape[0]*100

make_model                                 0.00
body_type                                  0.00
price                                      0.00
vat                                        0.00
km                                         0.00
Type                                       0.00
Cylinders                                  0.00
Fuel                                       0.00
Gears                                      0.00
Previous_Owners                            0.00
hp_kW                                      0.00
Inspection_new                             0.00
age                                        0.00
Body_Color                                 0.00
Paint_Type                                 0.00
Upholstery_type                            0.00
Nr_of_Doors                                0.00
Nr_of_Seats                                0.00
Gearing_Type                               0.00
Displacement_cc                            0.00
Weight_kg                               

In [159]:
df.to_csv("filled_scout_20200924.csv", index=False)